# Trading Manager for Financial

This notebook allow to comunicate with the services running in real time. All the comunications it is with MQ messages.

In [3]:
from src.infrastructure.database_handler import Universe
from src.application import conf
from src.domain.models.trading.petition import Petition
from src.domain.models.trading.order import Order
import datetime as dt
from src.infrastructure.brokerMQ import Emit_Events
import time
import math

# Conection to the database
store = Universe(host=conf.MONGO_HOST, port=conf.MONGO_PORT)
lib_petitions = store.get_library('petitions')
# Conection to broker mq
config_brokermq = {'host': conf.RABBITMQ_HOST, 'port': conf.RABBITMQ_PORT, 'user': conf.RABBITMQ_USER,
                   'password': conf.RABBITMQ_PASSWORD}
emiter = Emit_Events(config=config_brokermq)

def get_data_petition(name):
    for i in range(10):
        time.sleep(1)
        # check if name in list_symbols
        list_symbols = lib_petitions.list_symbols()
        if name in list_symbols:
            return lib_petitions.read(name).data
        time.sleep(5)



## Analice Realtime positions from the portfolio VS Realtime position in Exchange
It sends a petition msg to the portfolio service to get the realtime positions.

In [9]:
# Create the petition
name_to_saving = f'positions_{dt.datetime.now().strftime("%Y%m%d%H%M%S")}'
# name of portfolio which we want to know positions
name_portfolio = 'PortfolioForex1'
function_to_run='get_saved_values_strategies_last' #get_saved_values_strategy  
petition_pos = Petition(datetime=dt.datetime.now(), function_to_run=function_to_run,
                               name_to_saving=name_to_saving, name_portfolio=name_portfolio)

emiter.publish_event('petition', petition_pos)


In [10]:
# Get data Petition
data_petition = get_data_petition(name_to_saving)

## Agregate data_petition by ticker base
agregate = {}
tickers = []
for _id in data_petition.keys():
    p = data_petition[_id]
    t = data_petition[_id]['ticker']
    if t not in tickers:
       tickers.append(t)
    agregate.setdefault(t,0)
    agregate[t] += p['position']*p['quantity']
    agregate[t] = round(agregate[t], 2)

print('Porfolio Potitions')
list_currency = [c.split('-')[0] for c in agregate.keys() ]
agregate

Porfolio Potitions


{'USDSEK': 0.0, 'AUDNZD': -0.01, 'GBPUSD': -0.04, 'EURJPY': 0.0, 'EURNOK': 0.0}

In [11]:
# Get position in Broker or Exchange
broker = 'darwinex'
utc_now = dt.datetime.utcnow()
if broker == 'darwinex':
    _name_library = 'events_keeper'
    name_library = f'{_name_library}_{utc_now.strftime("%Y%m%d")}'
    lib_keeper = store.get_library(name_library)
    data = lib_keeper.read(f'{broker}_mt4_positions').data
   
else:
    pass

time_positions = data.datetime
diff_minutes = abs((utc_now-time_positions).seconds / 60)
if diff_minutes <1:
    broker_pos = data.positions
    print(broker_pos)
else:
    print('The saving positions is not updated')

{'GBPUSD': -0.04, 'AUDNZD': -0.01}


In [12]:
# Compute diference between broker and portfolio
diference = {}
for c in agregate.keys():
    if c not in broker_pos:
        quantity_broker = 0
    else:
        quantity_broker = broker_pos[c]
    diference[c]  = agregate[c] - quantity_broker
    if diference[c] > 0:
        diference[c] = round(diference[c],2)
    else:
        diference[c] = round(diference[c],2)
print(diference)

{'USDSEK': 0.0, 'AUDNZD': 0.0, 'GBPUSD': 0.0, 'EURJPY': 0.0, 'EURNOK': 0.0}


In [8]:
# Create order to close diference
n= 0
send = True  # by default is False
for t in tickers:
   d = diference[t.split('-')[0]]
   if abs(d) >= 0.01:
       if d > 0:
           action = 'buy'
       else:
           action = 'sell'
       order_id_sender =f' {0}_{n}_{dt.datetime.utcnow().strftime("%Y%m%d%H%M%S")}'
       order = Order(datetime=dt.datetime.utcnow(),
                            ticker=t, action=action,
                            price=0, quantity=abs(d), type='market', sender_id=0,
                            order_id_sender=order_id_sender)
       if send:
           emiter.publish_event('financial_order', order)
           n += 1

